In [1]:
import json
from yelpapi import YelpAPI
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:

with open('C:/Users/Shaun/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [3]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


In [4]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Sushi'


In [5]:
JSON_FILE = "Data/results_in_progress_NY_sushi.json"
JSON_FILE

'Data/results_in_progress_NY_sushi.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: if file_exists == False:
    
## CREATE ANY NEEDED FOLDERS# Get the Folder Name only
folder = os.path.dirname(JSON_FILE)
## If JSON_FILE included a folder:if len(folder)>0:
# create the folder
os.makedirs(folder,exist_ok=True)
        
        
## INFORM USER AND SAVE EMPTY LISTprint(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
# save an empty list
with open(JSON_FILE,'w') as f:
    json.dump([],f)  

# If it exists, inform userelse:
print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_sushi.json already exists.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results


4200

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

210

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists == True:
        # Check if user wants to delete if exists
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # delete file
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    
    # If it does NOT exist:
    else:
        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)
        
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

# Example usage:
# create_json_file("example.json", delete_if_exists=True)

In [14]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


210

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

# Exceeded 1000 api calls. Stopping loop.

  0%|          | 0/210 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,k8YJDry6_pbPIiCPeK-6mQ,mikado-brooklyn,Mikado,https://s3-media2.fl.yelpcdn.com/bphoto/MNIqGY...,False,https://www.yelp.com/biz/mikado-brooklyn?adjus...,44,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.69032888195518, 'longitude': -...","[delivery, pickup]","{'address1': '177 Atlantic Ave', 'address2': N...",+12123811388,(212) 381-1388,1681.307993,NaN
1,AudBWxeAr3zHr1ITrTcVpg,izakaya-fuku-jackson-heights,Izakaya Fuku,https://s3-media1.fl.yelpcdn.com/bphoto/mVURSC...,False,https://www.yelp.com/biz/izakaya-fuku-jackson-...,299,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.5,"{'latitude': 40.74635, 'longitude': -73.89383}","[pickup, delivery, restaurant_reservation]","{'address1': '71-28 Roosevelt Ave', 'address2'...",+17182551120,(718) 255-1120,9608.700043,$$
2,W0YGwDTYRjUCBgtpqHXSAg,iwak-brooklyn,IWak,https://s3-media1.fl.yelpcdn.com/bphoto/JesScm...,False,https://www.yelp.com/biz/iwak-brooklyn?adjust_...,70,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",5.0,"{'latitude': 40.71244591155105, 'longitude': -...","[delivery, pickup]","{'address1': '340 Grand St', 'address2': None,...",+19299924925,(929) 992-4925,3301.811149,NaN
3,BPZfuPvSxCfoppU4lPFm0Q,domodomo-new-york-new-york-4,DOMODOMO - New York,https://s3-media1.fl.yelpcdn.com/bphoto/YX89PD...,False,https://www.yelp.com/biz/domodomo-new-york-new...,1596,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 40.7280299, 'longitude': -74.00167}",[delivery],"{'address1': '140 W Houston St', 'address2': N...",+16467070301,(646) 707-0301,2584.282555,$$$
4,pNe8ZCkGTe4cXfKdOF7kYQ,ise-restaurant-new-york-3,Ise Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/S5SGXw...,False,https://www.yelp.com/biz/ise-restaurant-new-yo...,642,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.7291024, 'longitude': -73.9900...","[restaurant_reservation, pickup, delivery]","{'address1': '63 Cooper Sq', 'address2': None,...",+12122284152,(212) 228-4152,2654.657530,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
995,6UbOi5oDDQUJAI9V2pv11w,akimoto-sushi-new-york,Akimoto Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/qPn2Ix...,False,https://www.yelp.com/biz/akimoto-sushi-new-yor...,105,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",2.5,"{'latitude': 40.7156982, 'longitude': -74.0071...","[pickup, delivery]","{'address1': '187 Church St', 'address2': '', ...",+12127663350,(212) 766-3350,1570.777813,$$
996,apIonqUbtLWLeZtzQ8FR8g,umi-kitchen-bayonne,Umi Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/Tnl04v...,False,https://www.yelp.com/biz/umi-kitchen-bayonne?a...,25,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 40.66901464160359, 'longitude': -...","[pickup, delivery]","{'address1': '639 Broadway', 'address2': 'Ste ...",+12014378888,(201) 437-8888,10888.686737,NaN
997,eultmfyScNW-KoYI0-m9uw,wow-sushi-rockville-centre,Wow Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/sXxnqV...,False,https://www.yelp.com/biz/wow-sushi-rockville-c...,46,"[{'alias': 'seafoodmarkets', 'title': 'Seafood...",4.5,"{'latitude': 40.6607388, 'longitude': -73.6290...",[delivery],"{'address1': '201 N Long Beach Rd', 'address2'...",+15166785730,(516) 678-5730,31210.651644,$$
998,skIF77R0hzf-a182aVk4gQ,geisha-asian-fusion-new-york,Geisha Asian Fusion,https://s3-media2.fl.yelpcdn.com/bphoto/qUKQ-o...,False,https://www.yelp.com/biz/geisha-asian-fusion-n...,49,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 40.84799, 'longitude': -73.93765}","[pickup, delivery]","{'address1': '4186 Broadway', 'address2': None...",+12129271246,(212) 927-1246,16550.191367,NaN
999,nsgnjPtxQTD2h-wPQWjuDA,asahi-sushi-norwood,Asahi Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/vMcUxS...,False,https://www.yelp.com/biz/asahi-sushi-norwood?a...,126,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 40.9990672651173, 'longitude': -7...","[pickup, delivery]","{'address1': '506 Livingston St', 'address2': ...",+12017847405,(201) 784-7405,32822.843870,$$


In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_Sushi.csv.gz', compression='gzip',index=False)